In [1]:
## Specify parameters and paths

dataPath = 'Z:/HarveyLab/Alan/Data/20171002/p3'
prb = 'poly2' # 'poly2' is shorthand for A1x32-poly2-5mm-...etc
UbuntuShareBase = 'C://Users/Alan/Desktop/UbuntuShare/'
clusteringPath = 'C://Users/Alan/Documents/Github/clustering-pipelines'
ms4Path = 'C:\\Users\\Alan\\Documents\\GitHub\\mountainlab\\packages\\mlpy\\mda'

In [2]:
import os
os.chdir(dataPath)
import sys
sys.path.append(clusteringPath)
import read_rhd as rhd
import numpy as np
import glob
import matplotlib.pyplot as plt
from shutil import copy2
import datetime
import subprocess
%matplotlib inline
## import tool to read mda files; mlpy is available on the ms4 github branch (https://github.com/magland/mountainlab/tree/ms4)
sys.path.append(ms4Path)
import mdaio

In [4]:
files = glob.glob('*.rhd')
#for x in files:
#    print(x)
files.sort(key=os.path.getmtime)  ## WARNING: this sorting may be OS sensitive, could use (key=lambda files: files[-10:-4]) instead
print('Sorted Files:')
for x in files:
    print(x)
    
basename = os.path.basename(dataPath) # taking foldername as basename
print('{0} is basename'.format(basename))
print('{0} files'.format(len(files)))

Sorted Files:
gridIndent_171002_180119.rhd
gridIndent_171002_180134.rhd
gridIndent_171002_180149.rhd
gridIndent_171002_180204.rhd
gridIndent_171002_180219.rhd
gridIndent_171002_180234.rhd
gridIndent_171002_180249.rhd
gridIndent_171002_180305.rhd
gridIndent_171002_180321.rhd
gridIndent_171002_180336.rhd
gridIndent_171002_180351.rhd
gridIndent_171002_180406.rhd
gridIndent_171002_180421.rhd
gridIndent_171002_180436.rhd
gridIndent_171002_180451.rhd
gridIndent_171002_180506.rhd
gridIndent_171002_180521.rhd
gridIndent_171002_180537.rhd
gridIndent_171002_180552.rhd
gridIndent_171002_180607.rhd
gridIndent_171002_180622.rhd
gridIndent_171002_180637.rhd
gridIndent_171002_180652.rhd
gridIndent_171002_180707.rhd
gridIndent_171002_180722.rhd
gridIndent_171002_180738.rhd
gridIndent_171002_180753.rhd
p3 is basename
27 files


In [ ]:
#batch write .rhd to large raw.mda

numFiles = len(files)

for i, file in enumerate(files):
    print('\n\n\nExtracting file {0} of {1}\n'.format(i, numFiles))
    print(file)
    d = rhd.read_rhd(file)
    
    
    if i == 0:
        recordings = d['amplifier_data']
    else:
        recordings = np.concatenate((recordings,d['amplifier_data']),axis=1)
    
    ##save digital input streams as individual files -- different recordings might have different #s of channels
    for digitalChannel in range(d['board_dig_in_data'].shape[0]):
        d['board_dig_in_data'][digitalChannel].tofile(os.path.splitext(file)[0] + 'chan' + str(digitalChannel) + '.di')
    ##save analog input streams as individual files
    for analogChannel in range(d['aux_input_data'].shape[0]):
        d['aux_input_data'][analogChannel].tofile(os.path.splitext(file)[0] + 'chan' + str(analogChannel) + '.ai')


if not os.path.exists(dataPath+'/alldata'):
    os.mkdir(dataPath+'/alldata')
mdaio.writemda16ui(recordings,dataPath+'/alldata/raw.mda')



# concatenate raw.mda file to UbuntuShare for mountainsort

In [5]:
## Copy data to Ubuntu share

dt = datetime.datetime.now()
yearMonthDay = 	dt.strftime('%Y%m%d')  
UbuntuSharePath = UbuntuShareBase+yearMonthDay+'/'+basename+'/'

if not os.path.exists(UbuntuSharePath):
    os.makedirs(UbuntuSharePath)
copy2(dataPath+'/alldata/raw.mda',UbuntuSharePath) ## copies raw.mda to UbuntuShare folder

'C://Users/Alan/Desktop/UbuntuShare/20171008/p3/raw.mda'

In [35]:
## define functions for writing and running shell script in Ubuntu virtualbox

def writeUbuntuShell(UbuntuSharePath,basename, probe=None, num_channels=32):
    '''
    UbuntuSharePath is path of path that is shared with Ubuntu Virtualbox
    basename is the string of the containing folder
    probe is shorthand for a probe: 'poly2' for A1x32-Poly2-5mm-50s-177-A32
    
    Returns: string of Ubuntu location of shell
    '''
    oldpath = os.getcwd()
    os.chdir(UbuntuSharePath)
    if probe == 'poly2': # if A1x32-Poly2-5mm-50s-177-A32
        copy2('C://Users/Alan/Documents/Github/clustering-pipelines/poly2Geom.csv',UbuntuSharePath)
        num_channels=32
    dt = datetime.datetime.now()
    yearMonthDay =     dt.strftime('%Y%m%d')  
    sortDir = '/home/alan/data/sortProjects/'+yearMonthDay
    sortDirFull = sortDir+'/datasets/'+basename ## make folder for sortProject with folder name as current date and basename as dataset
    rawMdaLoc = sortDirFull+'/raw.mda'

    with open('datasets%s.txt' % (yearMonthDay) ,'w+') as datasetFile:
        datasetFile.write(basename+' /datasets/'+basename+'\n')
    
    with open('pipelines'+yearMonthDay+'.txt','w+') as pipelineFile:
        pipelineFile.write('ms2alan ms2_alan.pipeline') # default mountainsort pipeline file
    
    with open('params%s.json' % (yearMonthDay) ,'w+') as paramFile:
        if probe == None:
            paramFile.write('{"samplerate":20000,"sign":-1}') # default sample rate is 20 kHz and usually see negative spikes
        elif probe == 'poly2':
            paramFile.write('{"samplerate":20000,"sign":-1,"adjacency_radius":100}')
    with open('tempShell.sh', 'w+', newline='\n') as shellfile: # newline=\n necessary for linux
        shellfile.write('#! /bin/bash -i\n') # including the -i makes the script interative and this allows for use of default PATH on SSH
        shellfile.write('mkdir -p '+sortDirFull+'\n')
        
        ## convert .dat to .mda, create prv, and copy to datasets
        shellfile.write('cd /media/sf_UbuntuShare/%s/%s\n' % (yearMonthDay,basename)) #directory of shared folder in Ubuntu Virtualbox
        #shellfile.write('/home/alan/mountainlab/bin/mdaconvert raw.dat raw.mda --dtype=uint16 --input_format=raw_timeseries --num_channels=%d\n' % (num_channels)) # convert .dat to .mda
        shellfile.write('/home/alan/mountainlab/bin/prv-create2 raw.mda\n')
        shellfile.write('cp /media/sf_UbuntuShare/%s/%s/raw.mda.prv %s\n' % (yearMonthDay,basename,sortDirFull))
        
        ## copy params.json file to sortDirFull
        
        shellfile.write('cp params%s.json %s/params.json\n' % (yearMonthDay,sortDirFull))
        
        if probe == 'poly2':
            shellfile.write('cp poly2Geom.csv %s/geom.csv\n' % (sortDirFull))

        ## copy datasets.txt and pipelines.txt to sortDir
        shellfile.write('cp datasets%s.txt %s/datasets.txt\n' % (yearMonthDay,sortDir))
        shellfile.write('cp pipelines%s.txt %s/pipelines.txt\n' % (yearMonthDay,sortDir))
        
        ## move to sortDir and run sorting project
        shellfile.write('cd %s\n' % (sortDir))
        shellfile.write('/home/alan/mountainlab/bin/kron-run ms2alan %s --_nodaemon\n' % (basename))
    os.chdir(oldpath)
    return '/media/sf_UbuntuShare/%s/%s/tempShell.sh' % (yearMonthDay, basename)

def runUbuntuShell(UbuntuShell):
    '''
    UbuntuShell is str with linux style directory of shell file
        this directory is returned from the writeUbuntuShell function
    For alan, that is something like /media/sf_UbuntuShare/DATE/DATABASE/shelltorun.sh
    '''
    with open('ubuntuRun.bat','w+') as file:
        ## change directory to that with share folder
        pathToVBOXmanage = '"C:/Program Files/Oracle/VirtualBox/VBoxManage.exe"'
        file.write('title run ubuntu shell\n')
        
        #####
        ## I've commented out the following bit of code because I set my vbox to always start upon login to Windows
        ## see https://hellojason.net/blog/automatically-launch-a-headless-virtualbox-vm-at-bootup/
        ## You can uncomment to auto startup your virtualbox
        ####
        
        ## start virtual box
        #file.write('call %s startvm "Ubuntu"\n' %(pathToVBOXmanage))
        ## i set up the virtualbox to set the RDONLYHOST property on setup
        ## this wait command waits for that property to change before proceeding
        #file.write('call %s guestproperty wait "Ubuntu" RDONLYHOST\n' %(pathToVBOXmanage))
        
        
        ## executes the script UbuntuShell
        file.write('call %s guestcontrol "Ubuntu" run --username alan --password 123ubuntubox --exe %s -- --text\n' % (pathToVBOXmanage, UbuntuShell))
        
    p = subprocess.Popen('ubuntuRun.bat',shell=True) #run the BAT file.
    stdout, stderr = p.communicate()
    # in jupyter, the stdout appears in the cmd window rather than the browser
    return stdout, stderr

In [36]:
ubuntuLocation = writeUbuntuShell(UbuntuSharePath,basename,probe=prb)

In [37]:
stdout, stderr = runUbuntuShell(ubuntuLocation)